In [6]:
#sender public and private key
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -outform DER -out send_priv.der
!openssl rsa -in private.pem -outform DER -pubout -out send_pub.der

'openssl' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
#reciver public and private key
!openssl genrsa -out private.pem 2048
!openssl rsa -in private.pem -outform DER -out rec_priv.der
!openssl rsa -in private.pem -outform DER -pubout -out rec_pub.der

In [ ]:
#AES key generation
!openssl rand 16 > aes.key

In [ ]:
from Crypto.Signature import pss
from Crypto.Hash import SHA256
from Crypto.PublicKey import RSA
from Crypto.Cipher import PKCS1_OAEP
from Crypto.Hash import SHA
from Crypto import Random


def sign_message(message,privkeyfile_der):
    key = RSA.import_key(open(privkeyfile_der,'rb').read())
    h = SHA256.new(message.encode())
    signature = pss.new(key).sign(h)
    return signature


def verify_sig(message,signature,pubkeyfile_der):
    key = RSA.import_key(open(pubkeyfile_der,'rb').read())
    h = SHA256.new(message)
    verifier = pss.new(key)
    try:
        verifier.verify(h, signature)
        print( "The signature is authentic.")
    except (ValueError, TypeError):
        print( "The signature is not authentic.")


def RSAencrypt(message,pubkeyfile_der):
    key = RSA.importKey(open(pubkeyfile_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)  ### Padding Scheme
    ciphertext = cipher.encrypt(message)
    return ciphertext

def RSAdecrypt(ciphertext,privatekey_der):
    key = RSA.importKey(open(privatekey_der,'rb').read())
    cipher = PKCS1_OAEP.new(key)
    message = cipher.decrypt(ciphertext)
    return message

In [ ]:
!echo -n "dog is cat and cat is dog" > msg

In [ ]:
#Using sender's private key and the plaintext file we create the signature 
#assign it to msg.sig(to be transmitted to the reciever) file

msg = open("msg").read()

signature = sign_message(msg,"send_priv.der")
try:
    fp = open("msg.sig","wb")
    fp.write(signature)
    fp.close()
except:
    print("IO error")

In [ ]:
#using AES-128bit in CTR mode we encrypt the plaintext file 
#assign it to msg.crypt file

!openssl aes-128-ctr -pbkdf2 -iv 101112131415161718191A1B1C1D1E1F -e -pass file:aes.key -in msg -out msg.crypt


In [ ]:
#Using recievers public key the sender can encrypt the aes key so to securely send it to the reciever 

msg = open("aes.key",'rb').read()

ct = RSAencrypt(msg,"rec_pub.der")
try:
    fp = open("symkey.crypt","wb")
    fp.write(ct)
    fp.close()
except:
    print("IO error")

In [ ]:
#decrypt symkey.crypt(RSA encrypted AES key) to produce symkey(AES key) file

ct1 = open("symkey.crypt", "rb").read()
symkey = RSAdecrypt(ct1,"rec_priv.der")

try:
    file = open("rec_aes.key", "wb")
    file.write(symkey)
    file.close()
except:
    print("IO error")

In [ ]:
#Using AES key(symkey) to decrypt msg.crypt(sent via sender) to msg(plaintext)

!openssl aes-128-ctr -pbkdf2 -iv 101112131415161718191A1B1C1D1E1F -d -pass file:rec_aes.key -in msg.crypt -out msg

In [ ]:
#Receiver verifies msg file signature using msg.sig(signature file)
#to make sure the content has not been tampered with

recsigfile= open("msg.sig", "rb").read()
rec_msg = open("msg", "rb").read()

verify_sig(rec_msg, recsigfile,"send_pub.der")